In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [6]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [7]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [8]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [9]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [10]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [11]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:              \
           Time   Ep  Ct    LOSS  PPL     NLL    KL   REG    LOSS         PPL   
62     3.448755    0  61  116.60  680  115.83  0.67  0.10  106.17  555.779785   
682    0.610842   10  61  113.36  568  112.85  0.50  0.01  105.49  535.566833   
1302   0.643712   20  61  112.69  549  111.98  0.70  0.01  105.09  524.617065   
1922   1.145984   30  61  112.28  538  111.40  0.87  0.01  104.89  519.143616   
2542   1.120090   40  61  112.00  530  111.00  1.00  0.01  104.72  513.733643   
3162   0.635580   50  61  111.78  524  110.68  1.09  0.01  104.60  509.855347   
3782   0.662918   60  61  111.61  520  110.43  1.17  0.00  104.52  507.568115   
4402   0.730040   70  61  111.46  516  110.21  1.24  0.00  104.47  506.258270   
5022   0.660210   80  61  111.34  513  110.03  1.30  0.00  104.44  505.581116   
5642   1.157673   90  61  111.26  511  109.91  1.34  0.01  104.56  508.690918   
6262   1.106902  100  61  111.18  509  109.80  1.38  0.01  104.51  507.626709   
6882   0.563320  110  61  111.11  507  109.69  1.41  0.01  104.46  506.601196   
7502   0.575285  120  61  111.04  505  109.59  1.45  0.01  104.42  505.582550   
8122   0.758908  130  61  110.99  504  109.50  1.48  0.01  104.32  502.927063   
8742   1.040268  140  61  110.93  502  109.41  1.51  0.01  104.30  502.231598   
9362   0.611055  150  61  110.87  501  109.33  1.54  0.01  104.25  500.904236   
9982   0.638563  160  61  110.83  500  109.25  1.56  0.01  104.25  500.823029   
10602  0.764321  170  61  110.78  498  109.18  1.59  0.01  104.21  499.777161   
11222  0.560877  180  61  110.73  497  109.11  1.62  0.01  104.21  499.815796   
11842  0.595586  190  61  110.69  496  109.04  1.64  0.01  104.19  498.862183   
12462  0.929402  200  61  110.65  495  108.98  1.66  0.01  104.18  498.936401   
13082  0.606808  210  61  110.61  494  108.92  1.68  0.01  104.16  498.042664   
13702  1.129375  220  61  110.59  494  108.88  1.70  0.01  104.23  500.276215   
14322  0.720128  230  61  110.56  493  108.83  1.72  0.01  104.18  498.322723   
14942  0.698811  240  61  110.53  492  108.79  1.73  0.01  104.20  499.155334   
15562  1.347637  250  61  110.50  491  108.75  1.75  0.01  104.16  497.754669   
16182  0.809624  260  61  110.47  491  108.71  1.76  0.01  104.18  498.705444   
16802  0.677558  270  61  110.45  490  108.67  1.77  0.01  104.18  498.708771   
17422  0.830242  280  61  110.42  489  108.63  1.79  0.01  104.19  498.697601   
18042  0.697569  290  61  110.40  489  108.59  1.80  0.01  104.19  498.675018   
...         ...  ...  ..     ...  ...     ...   ...   ...     ...         ...   
43462  0.620688  700  61  109.80  474  107.67  2.12  0.00  104.32  502.749146   
44082  0.680652  710  61  109.79  474  107.66  2.13  0.00  104.32  502.861847   
44702  0.641734  720  61  109.78  473  107.64  2.13  0.00  104.32  502.965912   
45322  0.631781  730  61  109.77  473  107.63  2.14  0.00  104.33  503.062347   
45942  0.625663  740  61  109.76  473  107.62  2.14  0.00  104.33  503.154938   
46562  0.627079  750  61  109.75  473  107.61  2.15  0.00  104.33  503.243713   
47182  0.692342  760  61  109.75  473  107.59  2.15  0.00  104.33  503.328674   
47802  0.630695  770  61  109.74  472  107.58  2.16  0.00  104.34  503.413147   
48422  0.625337  780  61  109.73  472  107.57  2.16  0.00  104.34  503.497192   
49042  0.629071  790  61  109.72  472  107.56  2.16  0.00  104.34  503.582916   
49662  0.621610  800  61  109.72  472  107.55  2.17  0.00  104.35  503.674164   
50282  0.632027  810  61  109.71  472  107.54  2.17  0.00  104.35  503.769043   
50902  0.626291  820  61  109.70  472  107.52  2.17  0.00  104.35  503.854065   
51522  0.883922  830  61  109.70  471  107.51  2.18  0.00  104.35  503.930969   
52142  0.619059  840  61  109.69  471  107.50  2.18  0.00  104.36  504.004974   
52762  0.938265  850  61  109.68  471  107.49  2.19  0.00  104.36  504.076843   
53382  0.680337  860  61  109.68  471  107.48  2.19  0.00

0 R: 1.000 P: 0.360 carry pockets room strap back work - space quality small
   1 R: 0.264 P: 0.102 pocket mouse power netbook ipad charger nice cord room usb
     11 R: 0.119 P: 0.119 sleeve protection inside air smell inch zipper protect neoprene pro
     12 R: 0.043 P: 0.043 ! love perfect price bought recommend perfectly inside color sleeve
   4 R: 0.099 P: 0.047 zipper months broke years 've year started strap weeks disappointed
     43 R: 0.052 P: 0.052 ; & pro retina quality ! big bought nice -
   5 R: 0.091 P: 0.025 ... 'm ! review made ? ca price thing time
     51 R: 0.040 P: 0.040 ! love color mac ordered recommend : cover buy pro
     52 R: 0.026 P: 0.026 gift bought loves amazon item christmas ordered ! received wife
   3 R: 0.186 P: 0.083 bottom top cover plastic back nice screen piece corners feet
     31 R: 0.104 P: 0.104 cover color keyboard pro mac love hard easy perfectly apple
{0: [1, 4, 3, 2], 1: [11, 13], 4: [43], 3: [31, 32], 2: [21]}


TRAIN:                           VALID:              \
           Time   Ep  Ct    LOSS  PPL     NLL    KL   REG    LOSS         PPL   
62     3.448755    0  61  116.60  680  115.83  0.67  0.10  106.17  555.779785   
682    0.610842   10  61  113.36  568  112.85  0.50  0.01  105.49  535.566833   
1302   0.643712   20  61  112.69  549  111.98  0.70  0.01  105.09  524.617065   
1922   1.145984   30  61  112.28  538  111.40  0.87  0.01  104.89  519.143616   
2542   1.120090   40  61  112.00  530  111.00  1.00  0.01  104.72  513.733643   
3162   0.635580   50  61  111.78  524  110.68  1.09  0.01  104.60  509.855347   
3782   0.662918   60  61  111.61  520  110.43  1.17  0.00  104.52  507.568115   
4402   0.730040   70  61  111.46  516  110.21  1.24  0.00  104.47  506.258270   
5022   0.660210   80  61  111.34  513  110.03  1.30  0.00  104.44  505.581116   
5642   1.157673   90  61  111.26  511  109.91  1.34  0.01  104.56  508.690918   
6262   1.106902  100  61  111.18  509  109.80  1.38  0.01  104.51  507.626709   
6882   0.563320  110  61  111.11  507  109.69  1.41  0.01  104.46  506.601196   
7502   0.575285  120  61  111.04  505  109.59  1.45  0.01  104.42  505.582550   
8122   0.758908  130  61  110.99  504  109.50  1.48  0.01  104.32  502.927063   
8742   1.040268  140  61  110.93  502  109.41  1.51  0.01  104.30  502.231598   
9362   0.611055  150  61  110.87  501  109.33  1.54  0.01  104.25  500.904236   
9982   0.638563  160  61  110.83  500  109.25  1.56  0.01  104.25  500.823029   
10602  0.764321  170  61  110.78  498  109.18  1.59  0.01  104.21  499.777161   
11222  0.560877  180  61  110.73  497  109.11  1.62  0.01  104.21  499.815796   
11842  0.595586  190  61  110.69  496  109.04  1.64  0.01  104.19  498.862183   
12462  0.929402  200  61  110.65  495  108.98  1.66  0.01  104.18  498.936401   
13082  0.606808  210  61  110.61  494  108.92  1.68  0.01  104.16  498.042664   
13702  1.129375  220  61  110.59  494  108.88  1.70  0.01  104.23  500.276215   
14322  0.720128  230  61  110.56  493  108.83  1.72  0.01  104.18  498.322723   
14942  0.698811  240  61  110.53  492  108.79  1.73  0.01  104.20  499.155334   
15562  1.347637  250  61  110.50  491  108.75  1.75  0.01  104.16  497.754669   
16182  0.809624  260  61  110.47  491  108.71  1.76  0.01  104.18  498.705444   
16802  0.677558  270  61  110.45  490  108.67  1.77  0.01  104.18  498.708771   
17422  0.830242  280  61  110.42  489  108.63  1.79  0.01  104.19  498.697601   
18042  0.697569  290  61  110.40  489  108.59  1.80  0.01  104.19  498.675018   
...         ...  ...  ..     ...  ...     ...   ...   ...     ...         ...   
43462  0.620688  700  61  109.80  474  107.67  2.12  0.00  104.32  502.749146   
44082  0.680652  710  61  109.79  474  107.66  2.13  0.00  104.32  502.861847   
44702  0.641734  720  61  109.78  473  107.64  2.13  0.00  104.32  502.965912   
45322  0.631781  730  61  109.77  473  107.63  2.14  0.00  104.33  503.062347   
45942  0.625663  740  61  109.76  473  107.62  2.14  0.00  104.33  503.154938   
46562  0.627079  750  61  109.75  473  107.61  2.15  0.00  104.33  503.243713   
47182  0.692342  760  61  109.75  473  107.59  2.15  0.00  104.33  503.328674   
47802  0.630695  770  61  109.74  472  107.58  2.16  0.00  104.34  503.413147   
48422  0.625337  780  61  109.73  472  107.57  2.16  0.00  104.34  503.497192   
49042  0.629071  790  61  109.72  472  107.56  2.16  0.00  104.34  503.582916   
49662  0.621610  800  61  109.72  472  107.55  2.17  0.00  104.35  503.674164   
50282  0.632027  810  61  109.71  472  107.54  2.17  0.00  104.35  503.769043   
50902  0.626291  820  61  109.70  472  107.52  2.17  0.00  104.35  503.854065   
51522  0.883922  830  61  109.70  471  107.51  2.18  0.00  104.35  503.930969   
52142  0.619059  840  61  109.69  471  107.50  2.18  0.00  104.36  504.004974   
52762  0.938265  850  61  109.68  471  107.49  2.19  0.00  104.36  504.076843   
53382  0.680337  860  61  109.68  471  107.48  2.19  0.00

0 R: 1.000 P: 0.366 carry pockets room strap back work space quality - small
   1 R: 0.259 P: 0.103 pocket mouse power netbook ipad charger cord nice room usb
     11 R: 0.104 P: 0.104 sleeve inside protection air inch smell protect zipper pro neoprene
     13 R: 0.051 P: 0.051 ! love perfect price recommend bought perfectly inside color sleeve
   4 R: 0.101 P: 0.049 zipper months broke years 've year strap started weeks buy
     43 R: 0.052 P: 0.052 ; & pro retina quality ! big bought perfectly -
   3 R: 0.094 P: 0.030 ... 'm review ! made ? ca price thing reviews
     31 R: 0.038 P: 0.038 ! love color mac ordered cover recommend : buy pro
     32 R: 0.026 P: 0.026 gift loves amazon bought item received christmas ! wife daughter
   2 R: 0.180 P: 0.079 bottom top cover plastic back nice screen piece corners feet
     21 R: 0.101 P: 0.101 cover color keyboard pro mac hard love easy apple perfectly
